# Core

In [ ]:
#| default_exp game.engine.core

In [ ]:
# | hide
%load_ext lab_black

This notebook contains the basic high-level types and helper methods used by other parts of the engine.

## Errors

In [ ]:
# | export
class GameError(RuntimeError):
    pass


class OptionError(RuntimeError):
    pass


class NoToken(GameError):
    pass


class RouteTooShort(GameError):
    pass


class RouteTooLong(GameError):
    pass


class ReusesCity(GameError):
    pass

## Logging

A game log that stores messages along with associated action IDs.

In [ ]:
# | export


class GameLog(list):
    def __init__(self, game):
        super().__init__()
        self.game = game

    def append(self, message):
        """Overrides the append method to add a log entry."""
        if not isinstance(message, GameLog.Entry):
            message = GameLog.Entry(message, self.game.current_action_id)
        super().append(message)

    class Entry:
        """A log entry storing a message and an action ID."""

        def __init__(self, message, action_id):
            self.message = message
            self.action_id = action_id

        def __repr__(self):
            """Representation of the log entry."""
            return f"<Entry message='{self.message}', action_id={self.action_id}>"

# Base Entities

## Assignable

In [ ]:
# | export
class Assignable:
    assignments = {}

    @classmethod
    def assigned(cls, assignable, key):
        return key in cls.assignments.get(assignable, {})

    @classmethod
    def assign(cls, assignable, key, value=True):
        if assignable not in cls.assignments:
            cls.assignments[assignable] = {}
        cls.assignments[assignable][key] = value

    @classmethod
    def remove_assignment(cls, assignable, key):
        if assignable in cls.assignments:
            cls.assignments[assignable].pop(key, None)

    @classmethod
    def remove_from_all(cls, assignables, key):
        for assignable in assignables:
            if cls.assigned(assignable, key):
                cls.remove_assignment(assignable, key)

## Entity

Intended to be inherited by all Game Entities.

In [ ]:
# | export
class Entity:
    def is_company(self):
        return False

    def is_corporation(self):
        return False

    def is_minor(self):
        return False

    def is_system(self):
        return False

    def is_operator(self):
        return False

    def is_player(self):
        return False

    def is_receivership(self):
        return False

    def is_share_pool(self):
        return False

    def is_closed(self):
        return False

## Item

In [ ]:
# | export


class Item:
    def __init__(self, description="", cost=0):
        self.description = description
        self.cost = cost

    def __eq__(self, other):
        return self.description == other.description and self.cost == other.cost

## Ownable

An interface inherited by all entities that can be owned. Contains an `owner` field and helper methods.

In [ ]:
# | export
class Ownable:
    def __init__(self):
        self._owner = None

    @property
    def owner(self):
        return self._owner

    @owner.setter
    def owner(self, value):
        self._owner = value

    def owned_by(self, entity):
        if not entity:
            return False
        return (
            self._owner == entity
            or getattr(self._owner, "owner", None) == entity
            or self._owner == getattr(entity, "owner", None)
        )

    def player(self):
        if hasattr(self._owner, "player") and callable(getattr(self._owner, "player")):
            return self._owner.player()
        return (
            self._owner
            if hasattr(self._owner, "player") and self._owner.player
            else None
        )

    def corporation(self):
        if self.corporation():
            return self
        return getattr(self._owner, "corporation", None)

    def owned_by_corporation(self):
        return hasattr(self._owner, "corporation") and callable(
            getattr(self._owner, "corporation")
        )

    def owned_by_player(self):
        return hasattr(self._owner, "player") and callable(
            getattr(self._owner, "player")
        )

    def is_corporation(self):
        return False

## Passer

Contains methods around passing. Should be inherited by anything that can act in the stock round.

In [ ]:
# | export
class Passer:
    def __init__(self):
        self._passed = False

    @property
    def passed(self):
        return self._passed

    @property
    def active(self):
        return not self._passed

    def pass_(self):
        self._passed = True

    def unpass(self):
        self._passed = False

## Share Price

In [ ]:
# | export
import re


class SharePrice:
    TYPE_MAP = {
        "p": "par",
        "e": "endgame",
        "c": "close",
        "b": "multiple_buy",
        "o": "unlimited",
        "y": "no_cert_limit",
        "l": "liquidation",
        "a": "acquisition",
        "r": "repar",
        "i": "ignore_one_sale",
        "j": "ignore_two_sales",
        "s": "safe_par",
        "P": "par_overlap",
        "x": "par_1",
        "z": "par_2",
        "w": "par_3",
        "C": "convert_range",
        "m": "max_price",
        "n": "max_price_1",
        "u": "phase_limited",
        "t": "type_limited",
        "B": "pays_bonus",
        "W": "pays_bonus_1",
        "X": "pays_bonus_2",
        "Y": "pays_bonus_3",
        "Z": "pays_bonus_4",
    }

    NON_HIGHLIGHT_TYPES = [
        "par",
        "safe_par",
        "par_1",
        "par_2",
        "par_3",
        "par_overlap",
        "safe_par",
        "convert_range",
        "max_price",
        "max_price_1",
        "repar",
        "type_limited",
    ]

    PAR_TYPES = ["par", "par_overlap", "par_1", "par_2", "par_3"]

    def __init__(
        self,
        code,
        row,
        column,
        unlimited_types=None,
        multiple_buy_types=None,
    ):
        m = re.match(r"(\d*)([a-zA-Z]*)", code)
        types = [self.TYPE_MAP[char] for char in m.group(2)]

        self.coordinates = (row, column)
        self.price = int(m.group(1))
        self.type = types[0] if types else None
        self.types = types or []
        self.corporations = []
        self.can_buy_multiple = self.type in (multiple_buy_types or [])
        self.limited = self.type not in (unlimited_types or [])

    def __eq__(self, other):
        return self.coordinates == other.coordinates

    @property
    def id(self):
        return f"{self.price},{','.join(map(str, self.coordinates))}"

    def counts_for_limit(self):
        return self.limited

    def buy_multiple(self):
        return self.can_buy_multiple

    def __str__(self):
        return f"{self.__class__.__name__} - {self.price} {self.coordinates}"

    def can_par(self):
        return self.type in self.PAR_TYPES

    def end_game_trigger(self):
        return self.type == "endgame"

    def liquidation(self):
        return self.type == "liquidation"

    def acquisition(self):
        return self.type == "acquisition"

    def highlight(self):
        return self.type and self.type not in self.NON_HIGHLIGHT_TYPES

    def normal_movement(self):
        return self.type != "liquidation"

    def remove_par(self):
        self.types = [t for t in self.types if t not in self.PAR_TYPES]
        self.type = self.types[0] if self.types else None

## Shareholder

In [ ]:
# | export
class ShareHolder:
    def __init__(self):
        self._shares_by_corporation = {}

    @property
    def shares(self):
        return [
            share for shares in self._shares_by_corporation.values() for share in shares
        ]

    @property
    def shares_by_corporation(self, sorted=False):
        if sorted:
            self._shares_by_corporation = dict(
                sorted(self._shares_by_corporation.items())
            )
        return self._shares_by_corporation

    def shares_of(self, corporation):
        return self._shares_by_corporation.get(corporation, [])

    def delete_share(self, share):
        self._shares_by_corporation.get(share.corporation, []).remove(share)

    def certs_of(self, corporation):
        return self.shares_of(corporation)

    def percent_of(self, corporation):
        shares = self._shares_by_corporation.get(corporation, [])
        return sum(share.percent for share in shares)

    def common_percent_of(self, corporation):
        shares = [
            share
            for share in self._shares_by_corporation.get(corporation, [])
            if not share.preferred
        ]
        return sum(share.percent for share in shares)

    def presidencies(self):
        return [
            corporation
            for corporation, shares in self._shares_by_corporation.items()
            if any(share.president for share in shares)
        ]

    def num_shares_of(self, corporation, ceil=True):
        percent = self.percent_of(corporation)
        num = percent / corporation.share_percent
        return int(num) if ceil else num

## Spender

In [ ]:
# | export


class Spender:
    def __init__(self):
        self.cash = 0

    def check_cash(self, amount, borrow_from=None):
        available = self.cash + (borrow_from.cash if borrow_from else 0)
        if (available - amount) < 0:
            raise GameError(f"{self.name} has {self.cash} and cannot spend {amount}")

    def check_positive(self, amount):
        if amount <= 0:
            raise GameError(f"{amount} is not valid to spend")

    def spend(
        self, cash, receiver, check_cash=True, check_positive=True, borrow_from=None
    ):
        if check_cash:
            self.check_cash(cash, borrow_from=borrow_from)
        if check_positive:
            self.check_positive(cash)

        # Check if we need to borrow from our borrow_from target
        if borrow_from and (cash > self.cash):
            amount_borrowed = cash - self.cash
            self.cash = 0
            borrow_from.cash -= amount_borrowed
        else:
            self.cash -= cash

        receiver.cash += cash

## Stock Movement

In [ ]:
# | export
class BaseMovement:
    def __init__(self, market):
        self.market = market

    def share_price(self, coordinates):
        row, column = coordinates
        return (
            self.market.market[row][column]
            if row < len(self.market.market) and column < len(self.market.market[row])
            else None
        )

    def left(self, corporation, coordinates):
        raise NotImplementedError

    def right(self, corporation, coordinates):
        raise NotImplementedError

    def down(self, corporation, coordinates):
        raise NotImplementedError

    def up(self, corporation, coordinates):
        raise NotImplementedError


class TwoDimensionalMovement(BaseMovement):
    def left(self, corporation, coordinates):
        r, c = coordinates
        if c > 0 and self.share_price([r, c - 1]):
            return [r, c - 1]
        else:
            return self.down(corporation, coordinates)

    def right(self, corporation, coordinates):
        r, c = coordinates
        if c + 1 >= len(self.market.market[r]):
            return self.up(corporation, coordinates)
        else:
            return [r, c + 1]

    def down(self, _corporation, coordinates):
        r, c = coordinates
        if r + 1 < len(self.market.market):
            r += 1
        return [r, c]

    def up(self, _corporation, coordinates):
        r, c = coordinates
        if r - 1 >= 0:
            r -= 1
        return [r, c]


class OneDimensionalMovement(BaseMovement):
    def left(self, _corporation, coordinates):
        r, c = coordinates
        if c - 1 >= 0:
            c -= 1
        return [r, c]

    def right(self, _corporation, coordinates):
        r, c = coordinates
        if c + 1 < len(self.market.market[r]):
            c += 1
        return [r, c]

    def down(self, corporation, coordinates):
        return self.left(corporation, coordinates)

    def up(self, corporation, coordinates):
        return self.right(corporation, coordinates)


class ZigZagMovement(BaseMovement):
    def __init__(self, market, ledge_movement):
        self.ledge_movement = ledge_movement
        super().__init__(market)

    def left(self, _corporation, coordinates):
        r, c = coordinates
        if self.ledge_movement:
            c -= 2
            c = 0 if c < 0 else c
        elif c - 2 >= 0:
            c -= 2
        return [r, c]

    def right(self, _corporation, coordinates):
        r, c = coordinates
        if self.ledge_movement:
            c += 2
            c = len(self.market.market[r]) - 1 if c >= len(self.market.market[r]) else c
        elif c + 2 < len(self.market.market[r]):
            c += 2
        return [r, c]

    def down(self, _corporation, coordinates):
        r, c = coordinates
        if c > 0:
            c -= 1
        return [r, c]

    def up(self, _corporation, coordinates):
        r, c = coordinates
        if c + 1 < len(self.market.market[r]):
            c += 1
        return [r, c]

## Stock Market

In [ ]:
# | export
class StockMarket:
    def __init__(
        self,
        market,
        unlimited_types,
        multiple_buy_types=None,
        zigzag=None,
        ledge_movement=None,
    ):
        self.par_prices = []
        self.has_close_cell = False
        self.zigzag = zigzag
        self.market = [
            [
                SharePrice(code, r_index, c_index, unlimited_types, multiple_buy_types)
                if code != ""
                else None
                for c_index, code in enumerate(row)
            ]
            for r_index, row in enumerate(market)
        ]

        for row in self.market:
            for price in row:
                if price and price.can_par:
                    self.par_prices.append(price)
                if price and price.type == "close":
                    self.has_close_cell = True

        self.par_prices.sort(
            key=lambda p: (p.price, p.coordinates[1], p.coordinates[0]), reverse=True
        )

        if self.zigzag:
            self.movement = ZigZagMovement(self, ledge_movement)
        elif self.one_d():
            self.movement = OneDimensionalMovement(self)
        else:
            self.movement = TwoDimensionalMovement(self)

    def one_d(self):
        return all(len(row) == 1 for row in self.market)

    def set_par(self, corporation, share_price):
        share_price.corporations.append(corporation)
        corporation.share_price = share_price
        corporation.par_price = share_price
        corporation.original_par_price = share_price

    def right_ledge(self, coordinates):
        row, col = coordinates
        return col + 1 == len(self.market[row])

    def move_right(self, corporation):
        coordinates = self.right(corporation, corporation.share_price.coordinates)
        self.move(corporation, coordinates)

    def right(self, corporation, coordinates):
        return self.movement.right(corporation, coordinates)

    def move_up(self, corporation):
        coordinates = self.up(corporation, corporation.share_price.coordinates)
        self.move(corporation, coordinates)

    def up(self, corporation, coordinates):
        return self.movement.up(corporation, coordinates)

    def move_down(self, corporation):
        coordinates = self.down(corporation, corporation.share_price.coordinates)
        self.move(corporation, coordinates)

    def down(self, corporation, coordinates):
        return self.movement.down(corporation, coordinates)

    def move_left(self, corporation):
        coordinates = self.left(corporation, corporation.share_price.coordinates)
        self.move(corporation, coordinates)

    def left(self, corporation, coordinates):
        return self.movement.left(corporation, coordinates)

    def find_share_price(self, corporation, directions):
        return self.find_relative_share_price(
            corporation.share_price, corporation, directions
        )

    def find_relative_share_price(self, share, corporation, directions):
        coordinates = share.coordinates
        price = self.share_price(coordinates)

        for direction in directions:
            if direction == "left":
                coordinates = self.left(corporation, coordinates)
            elif direction == "right":
                coordinates = self.right(corporation, coordinates)
            elif direction == "down":
                coordinates = self.down(corporation, coordinates)
            elif direction == "up":
                coordinates = self.up(corporation, coordinates)

            price = self.share_price(coordinates) or price

        return price

    def max_reached(self):
        return self.max_reached

    def move(self, corporation, coordinates, force=False):
        share_price = self.share_price(coordinates)

        if not share_price or share_price == corporation.share_price:
            return

        if not force and not share_price.normal_movement():
            return

        corporation.share_price.corporations.remove(corporation)
        corporation.share_price = share_price
        self.max_reached = True if share_price.end_game_trigger() else False
        share_price.corporations.append(corporation)

    def share_prices_with_types(self, types):
        return sorted(
            [
                price
                for row in self.market
                for price in row
                if price and any(t in types for t in price.types)
            ],
            key=lambda p: p.price,
            reverse=True,
        )

    def share_price(self, coordinates):
        row, column = coordinates

        if row < len(self.market) and column < len(self.market[row]):
            return self.market[row][column]

    def remove_par(self, price):
        self.par_prices.remove(price)
        price.remove_par()

# Game Flow

## Phase

In [ ]:
# | export


class Phase:
    def __init__(self, phases, game):
        self.index = 0
        self.phases = phases
        self.game = game
        self.depot = game.depot
        self.log = game.log
        self.setup_phase()

    def buying_train(self, entity, train, source):
        while train.sym in self.next_on:
            self.next_phase()

        self.game.rust_trains(train, entity)
        self.depot.depot_trains(clear=True)

        for event in train.events:
            getattr(self.game, f"event_{event['type']}")()
        train.events.clear()
        self.game.after_buying_train(train, source)

    @property
    def previous(self):
        return self.phases[: self.index]

    @property
    def current(self):
        return self.phases[self.index]

    @property
    def upcoming(self):
        return (
            self.phases[self.index + 1] if self.index + 1 < len(self.phases) else None
        )

    def train_limit(self, entity):
        if isinstance(self.train_limit, dict):
            return self.train_limit.get(entity.type, 0)
        else:
            return self.train_limit

    def available(self, phase_name):
        if not phase_name:
            return False
        index = next(
            (i for i, phase in enumerate(self.phases) if phase["name"] == phase_name),
            -1,
        )
        return index <= self.index

    def setup_phase(self):
        phase = self.phases[self.index]

        self.name = phase["name"]
        self.operating_rounds = phase.get("operating_rounds")
        self.train_limit = phase.get("train_limit")
        self.tiles = list(phase.get("tiles", []))
        self.events = phase.get("events", [])
        self.status = phase.get("status", [])
        self.corporation_sizes = phase.get("corporation_sizes")
        self.next_on = (
            list(self.phases[self.index + 1]["on"])
            if self.index + 1 < len(self.phases)
            else []
        )

        log_msg = f"-- Phase {self.name} ("
        if self.operating_rounds:
            log_msg += f"Operating Rounds: {self.operating_rounds} | "
        log_msg += f"Train Limit: {self.train_limit_to_str(self.train_limit)}"
        log_msg += f" | Available Tiles: {', '.join(map(str.capitalize, self.tiles))})"
        self.log.append(log_msg)
        self.trigger_events()

    def trigger_events(self):
        for company in self.game.companies:
            if company.owner:
                for ability in self.game.abilities(
                    company, "revenue_change", on_phase=self.name
                ):
                    company.revenue = ability.revenue

                for _ in self.game.abilities(company, "close", on_phase=self.name):
                    self.log.append(f"Company {company.name} closes")
                    company.close()

        for entity in self.game.companies + self.game.corporations:
            entity.remove_ability_when(self.name)

    def next_phase(self):
        self.index += 1
        self.setup_phase()

    def train_limit_to_str(self, train_limit):
        if isinstance(train_limit, dict):
            return ", ".join(f"{type}: {limit}" for type, limit in train_limit.items())
        else:
            return str(train_limit)

# Testing

In [ ]:
class TestTiles:
    TEST_TILES_HUMAN_READABLE = [
        {"tile": "45"},
        {"tile": "H11", "title": "1822PNW"},
        {"tile": "O8", "title": "1822PNW"},
        {"tile": "I12", "title": "1822PNW"},
        # open: https://github.com/tobymao/18xx/issues/5981
        {"tile": "H22", "title": "1828.Games"},
        # open: https://github.com/tobymao/18xx/issues/8178
        {"tile": "H18", "title": "1830", "fixture": "26855", "action": 385},
        {"tile": "C15", "title": "1846"},
        # open: https://github.com/tobymao/18xx/issues/5167
        {"tile": "N11", "title": "1856", "fixture": "hotseat005", "action": 113},
        {"tile": "L0", "title": "1868 Wyoming"},
        {"tile": "WRC", "title": "1868 Wyoming"},
        {"tile": "F12", "title": "1868 Wyoming", "fixture": "1868WY_5", "action": 835},
        {"tile": "L0", "title": "1868 Wyoming", "fixture": "1868WY_5", "action": 835},
        {"tile": "J12", "title": "1868 Wyoming", "fixture": "1868WY_5", "action": 835},
        {"tile": "J12", "title": "1868 Wyoming", "fixture": "1868WY_5"},
        # open: https://github.com/tobymao/18xx/issues/4992
        {"tile": "I11", "title": "1882", "fixture": "5236", "action": 303},
        # open: https://github.com/tobymao/18xx/issues/6604
        {"tile": "L41", "title": "1888"},
        # open: https://github.com/tobymao/18xx/issues/5153
        {"tile": "IR7", "title": "18Ireland"},
        {"tile": "IR8", "title": "18Ireland"},
        # open: https://github.com/tobymao/18xx/issues/5673
        {"tile": "D19", "title": "18Mag", "fixture": "hs_tfagolvf_76622"},
        {"tile": "I14", "title": "18Mag", "fixture": "hs_tfagolvf_76622"},
        # open: https://github.com/tobymao/18xx/issues/7765
        {"tile": "470", "title": "18MEX"},
        {"tile": "475", "title": "18MEX"},
        {"tile": "479P", "title": "18MEX"},
        {"tile": "485P", "title": "18MEX"},
        {"tile": "486P", "title": "18MEX"},
    ]

    TEST_TILES = {}
    for opts in TEST_TILES_HUMAN_READABLE:
        tile = opts["tile"]
        title = opts.get("title", "DefaultTitle")
        fixture = opts.get("fixture", "DefaultFixture")
        action = opts.get("action", "DefaultAction")

        if title not in TEST_TILES:
            TEST_TILES[title] = {}

        if fixture not in TEST_TILES[title]:
            TEST_TILES[title][fixture] = {}

        if action not in TEST_TILES[title][fixture]:
            TEST_TILES[title][fixture][action] = []

        TEST_TILES[title][fixture][action].append([tile, opts])

In [ ]:
TestTiles.TEST_TILES

{'DefaultTitle': {'DefaultFixture': {'DefaultAction': [['45',
     {'tile': '45'}]]}},
 '1822PNW': {'DefaultFixture': {'DefaultAction': [['H11',
     {'tile': 'H11', 'title': '1822PNW'}],
    ['O8', {'tile': 'O8', 'title': '1822PNW'}],
    ['I12', {'tile': 'I12', 'title': '1822PNW'}]]}},
 '1828.Games': {'DefaultFixture': {'DefaultAction': [['H22',
     {'tile': 'H22', 'title': '1828.Games'}]]}},
 '1830': {'26855': {385: [['H18',
     {'tile': 'H18', 'title': '1830', 'fixture': '26855', 'action': 385}]]}},
 '1846': {'DefaultFixture': {'DefaultAction': [['C15',
     {'tile': 'C15', 'title': '1846'}]]}},
 '1856': {'hotseat005': {113: [['N11',
     {'tile': 'N11',
      'title': '1856',
      'fixture': 'hotseat005',
      'action': 113}]]}},
 '1868 Wyoming': {'DefaultFixture': {'DefaultAction': [['L0',
     {'tile': 'L0', 'title': '1868 Wyoming'}],
    ['WRC', {'tile': 'WRC', 'title': '1868 Wyoming'}]]},
  '1868WY_5': {835: [['F12',
     {'tile': 'F12',
      'title': '1868 Wyoming',
    

# Publisher Info

In [ ]:
# | export

PUBLISHER_INFO = {
    "all_aboard_games": {
        "name": "All-Aboard Games",
        "url": "https://all-aboardgames.com/",
    },
    "deep_thought_games": {
        "name": "Deep Thought Games",
        "url": "https://boardgamegeek.com/boardgamepublisher/4192/deep-thought-games-llc",
        "hidden": True,
    },
    "gmt_games": {
        "name": "GMT Games",
        "url": "https://www.gmtgames.com/",
    },
    "golden_spike": {
        "name": "Golden Spike Games",
        "url": "https://goldenspikegames.com/",
    },
    "grand_trunk_games": {
        "name": "Grand Trunk Games",
        "url": "https://www.grandtrunkgames.com/",
    },
    "lonny_games": {
        "name": "Lonny Games",
        "url": "https://www.lonnygames.com/",
    },
    "lookout": {
        "name": "Lookout Games",
        "url": "https://lookout-spiele.de/",
        "hidden": True,
    },
    "loserdogs": {
        "name": "Loserdogs",
        "url": "http://tanisan.com/ld/",
        "hidden": True,
    },
    "marflow_games": {
        "name": "Marflow Games",
        "url": "https://18xx-marflow-games.de/",
    },
    "mayfair": {
        "name": "Mayfair Games",
        "url": "https://boardgamegeek.com/boardgamepublisher/10/mayfair-games",
        "hidden": True,
    },
    "mercury": {
        "name": "Mercury Games",
        "url": "https://www.mercurygames.com/",
        "hidden": True,
    },
    "oo_games": {
        "name": "Double-O Games",
        "url": "http://ohley.de/english/",
        "hidden": True,
    },
    "seahorse": {
        "name": "Seahorse Laser & Design",
        "url": "https://www.etsy.com/shop/SeahorseLaserDesign?section_id=24360565",
        "hidden": True,
    },
    "self_published": {
        "name": "Self-published",
        "url": "https://18xx.games",
        "hidden": True,
    },
    "traxx": {
        "name": "TraXX",
        "url": "https://traxx-denver.com/games/",
    },
    "zman_games": {
        "name": "Z-MAN Games",
        "url": "https://zmangames.com/",
        "hidden": True,
    },
}

In [ ]:
PUBLISHER_INFO

{'all_aboard_games': {'name': 'All-Aboard Games',
  'url': 'https://all-aboardgames.com/'},
 'deep_thought_games': {'name': 'Deep Thought Games',
  'url': 'https://boardgamegeek.com/boardgamepublisher/4192/deep-thought-games-llc',
  'hidden': True},
 'gmt_games': {'name': 'GMT Games', 'url': 'https://www.gmtgames.com/'},
 'golden_spike': {'name': 'Golden Spike Games',
  'url': 'https://goldenspikegames.com/'},
 'grand_trunk_games': {'name': 'Grand Trunk Games',
  'url': 'https://www.grandtrunkgames.com/'},
 'lonny_games': {'name': 'Lonny Games', 'url': 'https://www.lonnygames.com/'},
 'lookout': {'name': 'Lookout Games',
  'url': 'https://lookout-spiele.de/',
  'hidden': True},
 'loserdogs': {'name': 'Loserdogs',
  'url': 'http://tanisan.com/ld/',
  'hidden': True},
 'marflow_games': {'name': 'Marflow Games',
  'url': 'https://18xx-marflow-games.de/'},
 'mayfair': {'name': 'Mayfair Games',
  'url': 'https://boardgamegeek.com/boardgamepublisher/10/mayfair-games',
  'hidden': True},
 'me